## basic workings

In [ ]:
!pip install 

     |████████████████████████████████| 2.6 MB 13.2 MB/s 
     |████████████████████████████████| 3.3 MB 46.7 MB/s 
     |████████████████████████████████| 636 kB 52.7 MB/s 
     |████████████████████████████████| 895 kB 47.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import numpy as np

from transformers import AutoTokenizer, AutoModel, pipeline

import torch
from torch.nn import functional as F

In [ ]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Biocreative/Biocreative/litcovid_dataset.csv')

In [ ]:
sample_row = data.iloc[]
sample_row

pmid                                                             32661141
journal                                                           J Virol
title                   Metabolic Syndrome and Viral Pathogenesis: Les...
abstract                Metabolic syndrome increases the risk of sever...
keywords                covid-19;sars-cov-2;diabetes;influenza;metabol...
pub_type                Journal Article;Research Support, N.I.H., Extr...
authors                 Smith, Maria;Honce, Rebekah;Schultz-Cherry, St...
doi                                                  10.1128/JVI.00665-20
label                                                 Treatment;Mechanism
Case Report                                                             0
Diagnosis                                                               0
Epidemic Forecasting                                                    0
Mechanism                                                               1
Prevention                            

In [ ]:
sample_item = sample_row['abstract']
sample_labels = sample_row.to_dict()
labels = list(sample_labels.keys())[9:]
print(labels, sample_item)

['Case Report', 'Diagnosis', 'Epidemic Forecasting', 'Mechanism', 'Prevention', 'Transmission', 'Treatment'] Metabolic syndrome increases the risk of severe disease due to viral infection. Yet few studies have assessed the pathogenesis of respiratory viruses in high-risk populations. Here, we summarize how metabolic dysregulation impairs immune responses, and we define the role of metabolism during influenza virus and coronavirus infections. We also discuss the use of various in vitro, in vivo, and ex vivo models to elucidate the contributions of host factors to viral susceptibility, immunity, and disease severity.


In [ ]:
classifier(sample_item, labels)

{'labels': ['Mechanism',
  'Diagnosis',
  'Case Report',
  'Transmission',
  'Epidemic Forecasting',
  'Prevention',
  'Treatment'],
 'scores': [0.24647271633148193,
  0.1775263249874115,
  0.15522360801696777,
  0.12992294132709503,
  0.12695592641830444,
  0.0996629074215889,
  0.06423550099134445],
 'sequence': 'Metabolic syndrome increases the risk of severe disease due to viral infection. Yet few studies have assessed the pathogenesis of respiratory viruses in high-risk populations. Here, we summarize how metabolic dysregulation impairs immune responses, and we define the role of metabolism during influenza virus and coronavirus infections. We also discuss the use of various in vitro, in vivo, and ex vivo models to elucidate the contributions of host factors to viral susceptibility, immunity, and disease severity.'}

In [ ]:
MODEL_NAME = 'deepset/sentence_bert'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

Some weights of the model checkpoint at deepset/sentence_bert were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
data = pd.read_csv('drive/MyDrive/Biocreative/Biocreative/litcovid_dataset.csv')
data = data.drop(['pmid', 'journal', 'title', 'keywords', 'pub_type', 'authors', 'doi', 'label'], axis=1)
data.head()

,abstract,Case Report,Diagnosis,Epidemic Forecasting,Mechanism,Prevention,Transmission,Treatment
0,"In December 2019, a new and highly contagious ...",0,0,0,1,0,0,1
1,"A novel coronavirus disease (COVID-19), transm...",0,0,0,0,1,0,1
2,"BACKGROUND: In December 2019, the novel corona...",1,0,0,0,0,0,0
3,During the coronavirus disease 2019 (COVID-19)...,0,0,0,0,1,0,0
4,OBJECTIVES: Sofosbuvir and daclatasvir are dir...,0,0,0,0,0,0,1


In [ ]:
LABEL_COLUMNS = data.columns[1:]
Sample_row = data.iloc[1]
Sample_abstract = Sample_row.abstract
Sample_labels = Sample_row[LABEL_COLUMNS]

In [ ]:
true_labels = [key for key, value in Sample_labels.to_dict().items() if value == 1]
true_labels

['Prevention', 'Treatment']

In [ ]:
Sample_abstract

'A novel coronavirus disease (COVID-19), transmitted from humans to humans, has rapidly become the pandemic responsible for the current global health crisis. COVID-19 is caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2), which is said to be of zoonotic origin. This review describes the etiology and signs and symptoms as well as the current allopathic therapy for COVID-19. Additionally, findings of previous studies on the immunomodulatory effects and antiviral activities of particular foods and herbs on influenza virus and coronaviruses have been collated, with the aim of promoting the use of dietary therapy and herbal medicine as COVID-19 preventive therapies, while specific drugs and vaccines are yet to be discovered or are still under development. The volume of existing reports is irrefutable evidence that foods and herbs possess a potential antiviral ability against SARS-CoV-2 and can prevent COVID-19. Foods and herbs could be used as dietary or complementary th

In [ ]:
encoding = tokenizer.batch_encode_plus(
    [Sample_abstract] + true_labels,
    return_tensors="pt",
    pad_to_max_length=True
) 

encoding.keys()

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
output = model(encoding["input_ids"], encoding["attention_mask"])[0]

In [ ]:
sentence_rep = output[:1].mean(dim=1)
label_reps = output[1:].mean(dim=1)

In [ ]:
similarities = F.cosine_similarity(sentence_rep, label_reps)
closest = similarities.argsort(descending=True)
for ind in closest:
    print(f'label: {true_labels[ind]} \t similarity: {similarities[ind]}')

label: Treatment 	 similarity: 0.24514493346214294
label: Prevention 	 similarity: 0.17372678220272064


## main model

In [1]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 32.2 MB/s 
     |████████████████████████████████| 895 kB 38.0 MB/s 
     |████████████████████████████████| 636 kB 42.4 MB/s 
     |████████████████████████████████| 3.3 MB 38.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip install pytorch-lightning

     |████████████████████████████████| 915 kB 32.4 MB/s 
     |████████████████████████████████| 5.6 MB 48.9 MB/s 
     |████████████████████████████████| 118 kB 55.3 MB/s 
     |████████████████████████████████| 829 kB 33.8 MB/s 
     |████████████████████████████████| 234 kB 49.7 MB/s 
     |████████████████████████████████| 1.3 MB 42.2 MB/s 
     |████████████████████████████████| 294 kB 51.2 MB/s 
     |████████████████████████████████| 142 kB 59.0 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=de8a46483367084e101aaeefd92e84ae0a4f6f4d1178afb5b9636ec38c98cac9
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninstalled tensorboard-2.5.0
  Attempting uninstall: future
    Found existing installation: future 0.16.0
 

In [3]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch import optim

from transformers import AutoTokenizer, AutoModel, AdamW, get_linear_schedule_with_warmup

import pytorch_lightning as pl
from pytorch_lightning.metrics.functional import accuracy, f1, auroc
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

%matplotlib inline  
%config InlineBackend.figure_format='retina'

RANDOM_SEED = 42

sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8

pl.seed_everything(RANDOM_SEED)

Global seed set to 42


42

In [4]:
df = pd.read_csv('drive/MyDrive/Biocreative/Biocreative/litcovid_pro.csv')
df.head()

,abstract,Case Report,Diagnosis,Epidemic Forecasting,Mechanism,Prevention,Transmission,Treatment
0,"In December 2019, a new and highly contagious ...",0,0,0,1,0,0,1
1,"A novel coronavirus disease (COVID-19), transm...",0,0,0,0,1,0,1
2,"BACKGROUND: In December 2019, the novel corona...",1,0,0,0,0,0,0
3,During the coronavirus disease 2019 (COVID-19)...,0,0,0,0,1,0,0
4,OBJECTIVES: Sofosbuvir and daclatasvir are dir...,0,0,0,0,0,0,1


In [5]:
train_df, val_df = train_test_split(df, test_size=0.1)
train_df.shape, val_df.shape

((22464, 8), (2496, 8))

In [6]:
LABEL_COLUMNS = df.columns.tolist()[1:]

In [7]:
BERT_MODEL_NAME = 'dmis-lab/biobert-base-cased-v1.1'
tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL_NAME)

### test tokenizer 

In [8]:
sample_item = train_df.abstract[0]
print(sample_item)
encoding = tokenizer.encode_plus(
    sample_item,
    add_special_tokens=True,
    max_length=511,
    return_token_type_ids=False,
    padding="max_length",
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt',
)
encoding.keys()

In December 2019, a new and highly contagious infectious disease emerged in Wuhan, China. The etiologic agent was identified as a novel coronavirus, now known as Severe Acute Syndrome Coronavirus-2 (SARS-CoV-2). Recent research has revealed that virus entry takes place upon the union of the virus S surface protein with the type I transmembrane metallo-carboxypeptidase, angiotensin converting enzyme 2 (ACE-2) identified on epithelial cells of the host respiratory tract. Virus triggers the synthesis and release of pro-inflammatory cytokines, including IL-6 and TNF-alpha and also promotes downregulation of ACE-2, which promotes a concomitant increase in levels of angiotensin II (AT-II). Both TNF-alpha and AT-II have been implicated in promoting overexpression of tissue factor (TF) in platelets and macrophages. Additionally, the generation of antiphospholipid antibodies associated with COVID-19 may also promote an increase in TF. TF may be a critical mediator associated with the developmen

dict_keys(['input_ids', 'attention_mask'])

In [9]:
encoding["input_ids"].shape, encoding["attention_mask"].shape

(torch.Size([1, 511]), torch.Size([1, 511]))

In [10]:
# encoding labels
sample = train_df.iloc[0]
label_dict = sample[1:].to_dict()
true_labels = [key for key, value in label_dict.items() if value == 1]
print(true_labels)
# see something for multiple labels

['Prevention']


In [11]:
label_encoding = tokenizer.encode_plus(
    true_labels[0],
    add_special_tokens=False,
    return_tensors="pt",
    return_token_type_ids=False
)
label_encoding.keys()

dict_keys(['input_ids', 'attention_mask'])

In [12]:
label_encoding["input_ids"], tokenizer.convert_ids_to_tokens(label_encoding["input_ids"])

(tensor([[13347]]), ['prevention'])

In [13]:
abstract_input = encoding["input_ids"]
label_input = label_encoding["input_ids"]
input_tensor = torch.cat((abstract_input, label_input), 1)
print(f"abstract input encoding shape {abstract_input.shape}")
print(f"label input encoding shape {label_input.shape}")
print(f"resultant input encoding shape {input_tensor.shape}")

abstract input encoding shape torch.Size([1, 511])
label input encoding shape torch.Size([1, 1])
resultant input encoding shape torch.Size([1, 512])


In [14]:
# same goes with attention mask
attention_tensor = torch.cat((encoding["attention_mask"], label_encoding["attention_mask"]), 1)
print(attention_tensor.shape)

torch.Size([1, 512])


In [15]:
# initialize bert model
bert_model = AutoModel.from_pretrained(BERT_MODEL_NAME)

Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.1 were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
output = bert_model(input_tensor, attention_tensor)
print(output.last_hidden_state.shape, output.pooler_output.shape)

torch.Size([1, 512, 768]) torch.Size([1, 768])


In [17]:
# fully connected layer
fc_layer = nn.Linear(bert_model.config.hidden_size, 1)

In [18]:
Output = fc_layer(output.pooler_output)
Output

tensor([[-0.0257]], grad_fn=<AddmmBackward>)

In [19]:
# define criterion
criterion = nn.BCELoss()

In [20]:
truth_value = torch.Tensor([1])
truth_value.shape

torch.Size([1])

In [21]:
torch.sigmoid(Output)

tensor([[0.4936]], grad_fn=<SigmoidBackward>)

In [22]:
truth_value = torch.Tensor([1])

In [23]:
loss = criterion(torch.sigmoid(Output), truth_value.view(1, 1))
loss

### continuation